In [ ]:
from sklearn.metrics import classification_report
import numpy as np
import os
import cv2

In [ ]:
def evaluate(net1_anno_addr,net1_output_addr):
    labels =np.sort(os.listdir(net1_anno_addr))

    masks_list,pre_l = [], []


    for i in range(len(labels)):
        masks_list.append(cv2.imread(net1_anno_addr+labels[i])[:,:,0])
        pre_l.append(np.floor(cv2.imread(net1_output_addr+labels[i])[:,:,2]/200))

    masks_np = np.asarray(masks_list)
    pre_np = np.asarray(pre_l)

    y_train = np.asarray(masks_np, dtype=np.float32)
    pre_np = np.asarray(pre_np, dtype=np.float32)

    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2], 1)
    yy = y_train.flatten()
    pre_np = pre_np.reshape(pre_np.shape[0], pre_np.shape[1], pre_np.shape[2], 1)
    pre_np = pre_np.flatten()
    print(pre_np.shape, yy.shape)
    #***************************************
    #***************************************
    print('performance :')
    print(classification_report(yy, pre_np, labels=[0,1], digits=4))

In [ ]:
def find_patches(output,labels_addr):
    reservoir_size = output.shape
    refine_patches,label = [], []
    patchx , patchy = 416,608
    labels=os.listdir(labels_addr)

    for patch_name in labels :

        numbers = []
        for word in patch_name.split():
            if word. isdigit():
                numbers.append(int(word))

        line, col = numbers[0], numbers[1]

        y0, y1 = (col-1)*patchy, col*patchy
        if y1 > reservoir_size[1] :
            y0, y1 = reservoir_size[1]-patchy, reservoir_size[1]

        x0,x1 = (line-1)*patchx, line*patchx
        if x1 > reservoir_size[0] :
            x0, x1 = reservoir_size[0]-patchx, reservoir_size[0]

        refine_patches.append(output[x0:x1 , y0:y1,0])
        label.append(cv2.imread(labels_addr+patch_name)[:,:,0])

    return refine_patches,label



def evaluate_using_mosaic(labels,mosaic):
    output = cv2.imread(mosaic)

    refine_patches,labels = find_patches(output,labels)
    refine = np.array(refine_patches).flatten()

    print('classification report: ')
    print(classification_report(np.array(labels).flatten(), refine, labels=[0,1], digits=4))

In [ ]:
def evaluate2(net2_patches_addr,net2_output):

    labels = os.listdir(net2_patches_addr+'anno/')

    masks_list,pre_l = [], []
    for i in range(len(labels)):
        masks_list.append(np.floor(cv2.imread(net2_patches_addr+'anno/'+labels[i])[:,:,0]/200))
        pre_l.append(cv2.imread(net2_output+labels[i])[:,:,0])


    masks_np = np.asarray(masks_list)
    pre_np = np.asarray(pre_l)

    y_train = np.asarray(masks_np, dtype=np.float32)
    pre_np = np.asarray(pre_np, dtype=np.float32)

    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], y_train.shape[2], 1)
    yy = y_train.flatten()
    pre_np = pre_np.reshape(pre_np.shape[0], pre_np.shape[1], pre_np.shape[2], 1)
    pre_np = pre_np.flatten()

    print('performance :')
    print(classification_report(yy, pre_np, labels=[0,1], digits=4))
